In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
model_id = 'codellama/CodeLlama-7b-Instruct-hf' #'deepseek-ai/deepseek-coder-1.3b-base'
model_id = 'deepseek-ai/deepseek-coder-1.3b-instruct'

In [3]:
# model = AutoModelForCausalLM.from_pretrained('deepseek-ai/deepseek-coder-1.3b-base')
model = AutoModelForCausalLM.from_pretrained(model_id)

In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()


In [5]:
device = 'cuda:0'
model = model.to(device)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

In [7]:
input_text = "Craft a Python function that takes a list of integer tuples representing coordinates and converts this list into a 2D NumPy array. The function should also normalize the array values between 0 and 1 based on the maximum value found in the entire list. Include thorough documentation within your code" 
inputs = tokenizer(input_text, return_tensors='pt')
inputs = inputs.to(device)

In [8]:
inputs

{'input_ids': tensor([[32013,    34,  3702,   245, 13003,  1155,   344,  4486,   245,  1517,
           280, 10878,   244,   393,   873, 14397, 13113,   285,  6145,  1267,
           437,  1517,   878,   245,   207,    17,    35, 20662, 23883,  3857,
            13,   428,  1155,  1020,   835,  4048,   938,   254,  3857,  3029,
          1433,   207,    15,   285,   207,    16,  2842,   331,   254,  6054,
          1432,  1496,   279,   254,  3903,  1517,    13,   680,  8049, 11350,
         12175,  2372,   518,  2974]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [9]:
#outputs = model.generate(input_ids = inputs['input_ids'], attention_mask = inputs['attention_mask'], max_new_tokens=100)

In [10]:
outputs = model.generate(**inputs, max_new_tokens=300)

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


In [11]:
outputs_ = tokenizer.batch_decode(outputs, skip_special_tokens = True)
output_text = outputs_[0]
# output_text = output_text[output_text.find('{'):output_text.rfind('}')+1]
# output_text = output_text[output_text.find('{'):output_text.rfind('}')+1]
print(output_text)

Craft a Python function that takes a list of integer tuples representing coordinates and converts this list into a 2D NumPy array. The function should also normalize the array values between 0 and 1 based on the maximum value found in the entire list. Include thorough documentation within your code.


def convert_to_numpy_array(coordinates):
    """
    This function takes a list of integer tuples representing coordinates and converts this list into a 2D NumPy array.
    It also normalizes the array values between 0 and 1 based on the maximum value found in the entire list.

    :param coordinates: List of integer tuples representing coordinates.
    :type coordinates: list
    :return: Normalized 2D NumPy array.
    :rtype: numpy.ndarray
    """

    # Convert the list of tuples into a 2D NumPy array
    numpy_array = np.array(coordinates)

    # Find the maximum value in the entire list
    max_value = np.max(numpy_array)

    # Normalize the array values between 0 and 1
    normaliz

In [12]:
import re

output_text = output_text.replace(input_text, '')
output_text = output_text[output_text.find('\ndef'):]
output_text =output_text.strip()
match = re.search( r'\n\S', output_text)
if match:
  output_text = output_text[:output_text.find(match.group())]
output_text = output_text.strip()
print(output_text)

def convert_to_numpy_array(coordinates):
    """
    This function takes a list of integer tuples representing coordinates and converts this list into a 2D NumPy array.
    It also normalizes the array values between 0 and 1 based on the maximum value found in the entire list.

    :param coordinates: List of integer tuples representing coordinates.
    :type coordinates: list
    :return: Normalized 2D NumPy array.
    :rtype: numpy.ndarray
    """

    # Convert the list of tuples into a 2D NumPy array
    numpy_array = np.array(coordinates)

    # Find the maximum value in the entire list
    max_value = np.max(numpy_array)

    # Normalize the array values between 0 and 1
    normalized_array = numpy_array / max_value

    return normalized_array


# Batching

In [92]:
input_texts = ["def fac():", 
               '#write a function for sum of 2 numbers\n\ndef sum():']
inputs = tokenizer(input_texts, 
                   padding=True,
                   return_tensors='pt')
inputs = inputs.to(device)

In [93]:
inputs

{'input_ids': tensor([[32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
         32014, 32014, 32013,  1551,  3535, 10942],
        [32013,     2,  6449,   245,  1155,   327,  2545,   280,   207,    17,
          5744,   185,   185,  1551,  2545, 10942]], device='cuda:0'), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [94]:
inputs['input_ids'].shape

torch.Size([2, 16])

In [99]:
outputs = model.generate(**inputs, max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


In [100]:
outputs = tokenizer.batch_decode(outputs, skip_special_tokens = True)
output_text = outputs
for text in output_text:
    print(text)

def fac():
    n = int(input())
    if n == 0:
        return 1
    else:
        return n * fac()


print(fac())
#write a function for sum of 2 numbers

def sum():
    a=int(input("enter the first number"))
    b=int(input("enter the second number"))
    c=a+b
    print(c)
sum()
